<a href="https://colab.research.google.com/github/beyounding/dyslexia_projects/blob/main/dyslexia_script_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algorithms for selecting exercises for the dyslexia intervention**

***Architecture***:

*   pre-processing of data after screening
*   algorithms select 5 exercises for reading difficulties
*   algorithms generate the text of the task

Input Sources:

*Classification of dyslexia and exercise errors*: R.I. Lalaeva, T.E. Egorov + classification of the original dataset proposed by the authors
Rello L, Baeza-Yates R, Ali A, Bigham JP, Serra M (2020) Predicting risk of dyslexia with an online gamified test. PLOS ONE 15(12): e0241687. https://doi.org/10.1371/journal.pone.0241687


*Screening Data*: https://www.kaggle.com/datasets/luzrello/dyslexia







**Data preprocessing**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/входной_скриннинг.csv') 

In [ ]:
for name, values in df.iteritems():
  df.loc[df[name] > 1, name] = 1

In [ ]:
df['F1'] = (df['F'] + df['F.2']+df['F.2']+df['F.3'])/3
df['S2'] = (df['S'] + df['S.1']+df['S.2']+df['S.3']+df['S.3']+df['S.4']+df['S.5']+df['S.6']+df['S.7']+df['S.8'])/10
df['F3'] = (df['F.4'] + df['F.5']+df['F.6']+df['F.7'])/4
df['S4'] = (df['S.9'] + df['S.10']+df['S.11']+df['S.12'])/4
df['F5'] = (df['F.8'] + df['F.9']+df['F.10']+df['F.11'])/4
df['S6'] = (df['S.13'] + df['S.14']+df['S.15'])/3
df['F7'] = (df['F.12'] + df['F.13'])/2
df['S8'] = (df['S.15'] + df['S.16'])/2
df.head()

,F,F.1,F.2,F.3,S,S.1,S.2,S.3,S.4,S.5,...,S.16,S.17,F1,S2,F3,S4,F5,S6,F7,S8
0,1.000000,1.000000,1.00,1.0,1.0,1.0,0.8,0.833333,0.8,1.0,...,0.114286,0.076923,1.333333,0.906667,1.000000,0.604167,0.819445,0.629630,0.058824,0.168254
1,1.000000,1.000000,1.00,0.0,1.0,0.0,1.0,1.000000,0.0,1.0,...,0.000000,0.115385,1.000000,0.666667,1.000000,0.000000,0.666667,0.311111,0.000000,0.166666
2,0.250000,0.250000,0.00,0.6,1.0,1.0,1.0,1.000000,1.0,0.0,...,0.000000,1.000000,0.283333,0.900000,1.000000,0.750000,0.625000,0.666667,0.525000,0.500000
3,1.000000,1.000000,1.00,1.0,1.0,0.8,1.0,1.000000,1.0,1.0,...,0.035088,0.115385,1.333333,0.930000,0.750000,0.725000,0.705357,0.231746,0.083334,0.100877
4,0.166667,0.222222,0.15,1.0,0.8,1.0,1.0,0.571429,1.0,1.0,...,0.088889,0.000000,0.488889,0.894286,0.770833,0.721429,0.760684,0.681159,0.416666,0.544444


In [ ]:
#iterate over the row values in the last total columns, the first letter is the error type. If F>S: to the Intervention add F column

df['TotalF'] = 0
df['TotalS'] = 0
count_f=0
count_s=0
for name, values in df[32:].iteritems():
    if str(name)[0]=='F':
      df['TotalF'] += df[name]
      count_f+=1
    elif str(name)[0]=='S':
      df['TotalS'] += df[name]
      count_s+=1
df['TotalF'] = df['TotalF']/count_f
df['TotalS'] = df['TotalS']/count_s

In [ ]:
#we look where the execution accuracy is less
df['compare']=df['TotalF']<df['TotalS']

#add as target of intervention
df['Intervention']=2
ind=0
for i in df['compare']:
  if i:
    df['Intervention'][ind]=1
  ind+=1

<ipython-input-6-5e575d308752>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Intervention'][ind]=1


In [ ]:
df.head()

,F,F.1,F.2,F.3,S,S.1,S.2,S.3,S.4,S.5,...,F3,S4,F5,S6,F7,S8,TotalF,TotalS,compare,Intervention
0,1.000000,1.000000,1.00,1.0,1.0,1.0,0.8,0.833333,0.8,1.0,...,1.000000,0.604167,0.819445,0.629630,0.058824,0.168254,0.811501,0.683582,False,2
1,1.000000,1.000000,1.00,0.0,1.0,0.0,1.0,1.000000,0.0,1.0,...,1.000000,0.000000,0.666667,0.311111,0.000000,0.166666,0.685185,0.357265,False,2
2,0.250000,0.250000,0.00,0.6,1.0,1.0,1.0,1.000000,1.0,0.0,...,1.000000,0.750000,0.625000,0.666667,0.525000,0.500000,0.615741,0.764394,True,1
3,1.000000,1.000000,1.00,1.0,1.0,0.8,1.0,1.000000,1.0,1.0,...,0.750000,0.725000,0.705357,0.231746,0.083334,0.100877,0.714451,0.637879,False,2
4,0.166667,0.222222,0.15,1.0,0.8,1.0,1.0,0.571429,1.0,1.0,...,0.770833,0.721429,0.760684,0.681159,0.416666,0.544444,0.607520,0.737765,True,1


In [ ]:
#we choose which of the input data we will select exercises. The index must be changed manually to access
# to the i row of the "Intervention" column

difficulty=df['Intervention'].iloc[0]

# **Selection of exercises**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
if difficulty==1:
  df = pd.read_csv("/content/Substitutions-and-mixing-sounds-when-reading.csv")
else:
  df = pd.read_csv("/content/Distortions-of-the-sound-syllabic-structure-of-the-word.csv")

In [ ]:
df['Exercise']=df['Exercise'].str.lower()
df['Intervention_Target']=df['Intervention_Target'].str.lower()
df['Reading_level']=df['Reading_level'].str.lower()

In [ ]:
df['Exercise']=df['Exercise'].str.replace('[^\w\s]','')
df['Intervention_Target']=df['Intervention_Target'].str.replace('[^\w\s]','')
df['Reading_level']=df['Reading_level'].str.replace('[^\w\s]','')

<ipython-input-15-6c0f458c9ee8>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Exercise']=df['Exercise'].str.replace('[^\w\s]','')
<ipython-input-15-6c0f458c9ee8>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Intervention_Target']=df['Intervention_Target'].str.replace('[^\w\s]','')
<ipython-input-15-6c0f458c9ee8>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Reading_level']=df['Reading_level'].str.replace('[^\w\s]','')


In [ ]:
df['Exercise']=df['Exercise'].str.strip()
df['Intervention_Target']=df['Intervention_Target'].str.strip()
df['Reading_level']=df['Reading_level'].str.strip()

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words = stopwords.words('english')
df['Exercise'] = df['Exercise'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df['Intervention_Target'] = df['Intervention_Target'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df['Reading_level'] = df['Reading_level'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
df.head()

,Index,Exercise,Reading_level,Intervention_Target
0,1,name vowel sound syllable,syllablebysyllable reading,distortions soundsyllabic structure word
1,2,record vowels syllables,syllablebysyllable reading,distortions soundsyllabic structure word
2,3,isolation sound background word,mastery soundletter designations,distortions soundsyllabic structure word
3,4,separation sound beginning end word,mastery soundletter designations,distortions soundsyllabic structure word
4,5,determining sequence sounds word,mastery soundletter designations,distortions soundsyllabic structure word


In [ ]:
features = ['Exercise', 'Intervention_Target', 'Reading_level']

In [ ]:
def combine_features(row):
    return row['Exercise']+' '+row['Intervention_Target']+' '+row['Reading_level']

In [ ]:
for feature in features:
    df[feature] = df[feature].fillna('')

df['combined_features'] = df.apply(combine_features, axis = 1)
print(df.loc[0, 'combined_features'])

name vowel sound syllable distortions soundsyllabic structure word syllablebysyllable reading


In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['combined_features'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix)
print(cosine_sim)

In [ ]:
#take the first exercise at random. It is assumed that here the specialist / the child himself, if he
#is able to realize the formation of his skill, notes whether such an exercise suits him
# if not, another one is selected. can be done through a conditional operator, but this is probably within the framework
#current research is redundant

first_ex = np.random.choice(df['Exercise'])
first_ex

'make syllables letters split alphabet'

In [ ]:
ex_user_likes = first_ex 
def get_index_from_title(title):
    return df[df.Exercise == title]["Index"].values[0]
ex_index = get_index_from_title(ex_user_likes)

In [ ]:
similar_ex = list(enumerate(cosine_sim[ex_index]))
sorted_similar_ex = sorted(similar_ex, key=lambda x:x[1], reverse=True)
print(sorted_similar_ex)

[(19, 1.0000000000000002), (14, 0.7205766921228922), (16, 0.7205766921228922), (18, 0.7205766921228922), (15, 0.6708203932499369), (17, 0.6301260378126043), (1, 0.4811252243246882), (0, 0.4564354645876385), (2, 0.4003203845127179), (4, 0.4003203845127179), (3, 0.38575837490522985), (6, 0.38575837490522985), (5, 0.3333333333333334), (13, 0.3333333333333334), (12, 0.3202563076101743), (7, 0.3086066999241839), (8, 0.3086066999241839), (10, 0.3086066999241839), (11, 0.29814239699997197), (9, 0.2886751345948129)]


In [ ]:
#displays 5 similar exercises
def get_title_from_index(index):
    return df[df.index == index]["Exercise"].values[0]
i=0
for ex in sorted_similar_ex:
    print(get_title_from_index(ex[0]))
    if i==0:
      docu_1 = get_title_from_index(ex[0])
    i=i+1
    if i>4:
        break

write syllables begin vowel
think words two three syllables
determine number syllables title picture
make syllables letters split alphabet
come word certain syllable beginning


In [ ]:
def jaccard_similarity(doc1, doc2):
 words_doc1 = set(doc1.lower().split())
 words_doc2 = set(doc2.lower().split())
 intersection = words_doc1.intersection(words_doc2)
 union = words_doc1.union(words_doc2)
 return float(len(intersection)) / len(union)

In [ ]:
#according to the second criterion similarity validation
#here you can use it as an additional criterion, or
# cycle through the exercises from the recommendations until the coefficient
#will not exceed the threshold - but it will complicate

print(jaccard_similarity(str(ex_user_likes), str(docu_1)))

0.125


**Task generation**

In [ ]:
"""""""""""""""""""""""""""""
We choose the current level of formation of reading skills in the child. Entered manually, because initial screening test
does not provide level information. Possible options:

reading_level='letter'
reading_level='syllable'
reading_level='word'
reading_level='phrase'
reading_level='sentence'

there is a corpus of text for assignments for each reader level. For letters - just an alphabet that is randomly generated,
for syllables - a corpus of the most frequent syllables, for words - a list of words, for phrases and sentences - full texts

in this version, I try it on the proposal, because this is the most difficult part, as if there was something coherent and adequate

"""""""""""""""""""""""""""""

reading_level='sentence'

In [ ]:
if reading_level=='sentence' or reading_level=='phrase':
  text = open('tol.txt', encoding='utf8').read()
#else:
   # open another file with the corpus
corpus = text.split()

In [ ]:
def make_pairs(corpus):
    for i in range(len(corpus)-1):
        yield (corpus[i], corpus[i+1])
pairs = make_pairs(corpus)

In [ ]:
word_dict = {}
for word_1, word_2 in pairs:
    if word_1 in word_dict.keys():
        word_dict[word_1].append(word_2)
    else:
        word_dict[word_1] = [word_2]

In [ ]:
first_word = np.random.choice(corpus)
while first_word.islower():
    first_word = np.random.choice(corpus)
chain = [first_word]

n_words = 8 #if a sentence is needed - it is better to put from 8, if the phrase is 2-3 words

for i in range(n_words):
    chain.append(np.random.choice(word_dict[chain[-1]]))
print(' '.join(chain))

Серёжа чижа в клетку и сетку оставил. Поглядел после
